# Einfaches Machine Learning

In diesem Notebook lernen wir die Grundlagen von **Machine Learning** (maschinellem Lernen) kennen.
Wir fangen ganz einfach an: mit einer Geraden, die durch Punkte gelegt wird.

---

## Teil 1: Lineare Regression

Stell dir vor, du hast ein paar Datenpunkte und möchtest eine Gerade finden, die möglichst gut durch diese Punkte geht.
Das nennt man **Lineare Regression**.

Eine Gerade wird durch zwei Zahlen beschrieben:
- **Steigung** (slope): Wie steil ist die Gerade?
- **Y-Achsenabschnitt** (intercept): Wo schneidet die Gerade die Y-Achse?

Die Formel lautet: $ y = \text{slope} \cdot x + \text{intercept} $

### Bibliotheken laden

Zuerst laden wir die Werkzeuge, die wir brauchen. Führe diese Zelle einmal aus.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact, FloatSlider
import ipywidgets as widgets

# Damit die Plots schön interaktiv sind
%matplotlib widget

### Unsere Datenpunkte

Wir erzeugen vier einfache Datenpunkte, die ungefähr auf einer Geraden liegen.

In [ ]:
# Unsere vier Datenpunkte
X = np.array([1, 2, 3, 4])       # X-Koordinaten
Y = np.array([2.4, 3.6, 6.6, 7.4])  # Y-Koordinaten (ungefähr y = 2*x, mit Streuung)

print("Unsere Datenpunkte:")
for i in range(len(X)):
    print(f"  Punkt {i+1}: x = {X[i]}, y = {Y[i]}")

### Was ist der "Loss"?

Wie gut passt unsere Gerade zu den Punkten? Das messen wir mit dem **Loss** (Verlust).

Für jeden Punkt berechnen wir:
1. Wo liegt der Punkt laut unserer Gerade? (Vorhersage)
2. Wo liegt er wirklich? (Wahrer Wert)
3. Wie groß ist der Unterschied? (Fehler)

Der **Mean Squared Error (MSE)** ist der Durchschnitt der quadrierten Fehler:

$ \text{Loss} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 $

Je kleiner der Loss, desto besser passt die Gerade!

In [ ]:
def berechne_loss(slope, intercept, X, Y):
    """Berechnet den Mean Squared Error für gegebene Parameter."""
    vorhersage = slope * X + intercept
    fehler = Y - vorhersage
    mse = np.mean(fehler ** 2)
    return mse

### Das Loss-Tal berechnen

Wir berechnen jetzt den Loss für viele verschiedene Kombinationen von slope und intercept.
Das ergibt eine 3D-Landschaft - das sogenannte **Loss-Tal**.

In [ ]:
# Wertebereiche für slope und intercept
slope_range = np.linspace(0, 4, 50)
intercept_range = np.linspace(-2, 4, 50)

# Gitter erstellen
SLOPE, INTERCEPT = np.meshgrid(slope_range, intercept_range)

# Loss für jeden Punkt im Gitter berechnen
LOSS = np.zeros_like(SLOPE)
for i in range(SLOPE.shape[0]):
    for j in range(SLOPE.shape[1]):
        LOSS[i, j] = berechne_loss(SLOPE[i, j], INTERCEPT[i, j], X, Y)

print("Loss-Tal berechnet!")
print(f"Minimaler Loss: {LOSS.min():.4f}")

### Interaktive Visualisierung

Jetzt kommt das Spannende! Mit den Schiebereglern kannst du slope und intercept verändern.

- **Links**: Die Datenpunkte und deine aktuelle Gerade
- **Rechts**: Das Loss-Tal mit deiner aktuellen Position (roter Punkt)

**Ziel**: Finde die Werte, bei denen der rote Punkt ganz unten im Tal liegt!

In [ ]:
# Figure erstellen
fig = plt.figure(figsize=(12, 5))

# Linker Plot: Datenpunkte und Gerade
ax1 = fig.add_subplot(1, 2, 1)
ax1.set_xlim(0, 5)
ax1.set_ylim(-2, 12)
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_title('Datenpunkte und Gerade')
ax1.grid(True, alpha=0.3)

# Datenpunkte zeichnen (bleiben fest)
ax1.scatter(X, Y, color='blue', s=100, zorder=5, label='Datenpunkte')

# Gerade (wird aktualisiert)
x_line = np.linspace(0, 5, 100)
line, = ax1.plot(x_line, x_line, 'r-', linewidth=2, label='Gerade')
ax1.legend()

# Rechter Plot: Loss-Tal (3D)
ax2 = fig.add_subplot(1, 2, 2, projection='3d')
surface = ax2.plot_surface(SLOPE, INTERCEPT, LOSS, cmap='coolwarm', alpha=0.8, edgecolor='none')
ax2.set_xlabel('Slope')
ax2.set_ylabel('Intercept')
ax2.set_zlabel('Loss')
ax2.set_title('Loss-Tal')

# Punkt-Projektion auf den Boden (Schatten) - SCHWARZ, zuerst zeichnen
bottom_point, = ax2.plot([2], [0], [0], 'ko', markersize=8, alpha=0.4)

# Verbindungslinie vom Schatten zum Punkt (gestrichelt)
drop_line, = ax2.plot([2, 2], [0, 0], [0, berechne_loss(2, 0, X, Y)], 'k--', linewidth=0.8, alpha=0.3)

# Aktueller Punkt im Loss-Tal (3D) - ROT, zuletzt zeichnen
current_point, = ax2.plot([2], [0], [berechne_loss(2, 0, X, Y)], 'ro', markersize=10, zorder=10)

# Loss-Anzeige als Text
loss_text = fig.text(0.5, 0.02, '', ha='center', fontsize=12, fontweight='bold')

plt.tight_layout()

def update(slope, intercept):
    """Aktualisiert die Plots wenn Schieberegler bewegt werden."""
    # Gerade aktualisieren
    y_line = slope * x_line + intercept
    line.set_ydata(y_line)
    
    # Aktuellen Loss berechnen
    current_loss = berechne_loss(slope, intercept, X, Y)
    
    # Punkt im Loss-Tal aktualisieren (rot, 3D)
    current_point.set_data_3d([slope], [intercept], [current_loss])
    
    # Verbindungslinie aktualisieren
    drop_line.set_data_3d([slope, slope], [intercept, intercept], [0, current_loss])
    
    # Projektion auf Boden aktualisieren (schwarz, 2D)
    bottom_point.set_data_3d([slope], [intercept], [0])
    
    # Loss-Text aktualisieren
    loss_text.set_text(f'Aktueller Loss: {current_loss:.4f}  |  slope = {slope:.2f}, intercept = {intercept:.2f}')
    
    fig.canvas.draw_idle()

# Schieberegler erstellen
slope_slider = FloatSlider(
    value=2.0,
    min=0.0,
    max=4.0,
    step=0.1,
    description='Slope:',
    continuous_update=True
)

intercept_slider = FloatSlider(
    value=0.0,
    min=-2.0,
    max=4.0,
    step=0.1,
    description='Intercept:',
    continuous_update=True
)

# Interaktive Widgets anzeigen
interact(update, slope=slope_slider, intercept=intercept_slider)

# Initiale Darstellung
update(2.0, 0.0)
plt.show()

---

### Aufgabe: Finde das Minimum!

Experimentiere mit den Schiebereglern und versuche, den kleinstmöglichen Loss zu finden.

1. Bei welchen Werten für **slope** und **intercept** ist der Loss am kleinsten?
2. Wie klein ist der minimale Loss, den du erreichen kannst?
3. Warum ist der Loss nicht genau 0? (Tipp: Schau dir die Datenpunkte an!)

**Deine Antworten:**

(Hier doppelklicken zum Bearbeiten)

- Bester slope: 
- Bester intercept: 
- Minimaler Loss: 
- Warum nicht 0?: 

---

### Bonus: Die mathematisch optimale Lösung

Es gibt eine Formel, die direkt die besten Werte berechnet. Das nennt man die **Normalengleichung**.
Vergleiche dein Ergebnis mit der mathematischen Lösung!

In [ ]:
# Optimale Lösung berechnen
n = len(X)
slope_optimal = (n * np.sum(X * Y) - np.sum(X) * np.sum(Y)) / (n * np.sum(X**2) - np.sum(X)**2)
intercept_optimal = (np.sum(Y) - slope_optimal * np.sum(X)) / n
loss_optimal = berechne_loss(slope_optimal, intercept_optimal, X, Y)

print(f"Optimale Lösung:")
print(f"  slope     = {slope_optimal:.4f}")
print(f"  intercept = {intercept_optimal:.4f}")
print(f"  Loss      = {loss_optimal:.4f}")

---

## Teil 2: Logistische Regression (Klassifikation)

Bei der linearen Regression haben wir Zahlenwerte vorhergesagt. Aber was, wenn wir entscheiden wollen, ob etwas zu **Kategorie 0** oder **Kategorie 1** gehört?

Das nennt man **Klassifikation**. Zum Beispiel:
- Ist diese E-Mail Spam (1) oder nicht (0)?
- Besteht der Schüler die Prüfung (1) oder nicht (0)?

Dafür nutzen wir die **logistische Regression**. Sie gibt uns eine Wahrscheinlichkeit zwischen 0 und 1.

### Die Sigmoid-Funktion

Das Herzstück ist die **Sigmoid-Funktion**. Sie verwandelt jeden Wert in eine Zahl zwischen 0 und 1:

$ \sigma(z) = \frac{1}{1 + e^{-z}} $

Wir berechnen $z$ aus dem Abstand zum **Mittelpunkt** und der **Steilheit** (wie schnell der Übergang von 0 zu 1 passiert):

$ z = \text{steilheit} \cdot (x - \text{mittelpunkt}) $

### Unsere Klassifikations-Daten

Wir erzeugen 20 zufällige Datenpunkte. Die Klasse wird mit einer Sigmoid-Funktion (Mitte bei 3, Steilheit 1) zufällig bestimmt - je weiter rechts ein Punkt liegt, desto wahrscheinlicher gehört er zu Klasse 1.

In [ ]:
def sigmoid(z):
    """Die Sigmoid-Funktion."""
    return 1 / (1 + np.exp(-z))

# Zufallsgenerator mit festem Seed (für Reproduzierbarkeit)
np.random.seed(42)

# 25 zufällige X-Werte zwischen 0 und 6
X_klass = np.random.uniform(0, 6, 25)
X_klass = np.sort(X_klass)  # Sortieren für bessere Übersicht

# Wahrscheinlichkeit für Klasse 1 mit Sigmoid (Mitte=3, Steilheit=1)
true_mitte = 3.0
true_steilheit = 3.0
p_klasse1 = sigmoid(true_steilheit * (X_klass - true_mitte))

# Labels zufällig samplen basierend auf der Wahrscheinlichkeit
Y_klass = (np.random.random(25) < p_klasse1).astype(int)

print("Unsere Klassifikations-Daten (25 Punkte):")
print(f"  Generiert mit: Mitte = {true_mitte}, Steilheit = {true_steilheit}")
print(f"\n  Links von x=3:  {sum(Y_klass[X_klass < 3])} von {sum(X_klass < 3)} sind Klasse 1")
print(f"  Rechts von x=3: {sum(Y_klass[X_klass > 3])} von {sum(X_klass > 3)} sind Klasse 1")

### Der Loss: Negative Log-Likelihood

Bei Klassifikation messen wir den Fehler anders. Wir nutzen die **Negative Log-Likelihood** (auch "Binary Cross-Entropy" genannt):

$ \text{Loss} = -\frac{1}{n} \sum_{i=1}^{n} \left[ y_i \cdot \log(p_i) + (1-y_i) \cdot \log(1-p_i) \right] $

Dabei ist $p_i$ die vorhergesagte Wahrscheinlichkeit für Klasse 1.

**Intuition:** 
- Wenn das Label 1 ist und wir $p = 0.99$ vorhersagen → kleiner Loss (gut!)
- Wenn das Label 1 ist und wir $p = 0.01$ vorhersagen → großer Loss (schlecht!)

In [ ]:
def berechne_log_loss(mittelpunkt, steilheit, X, Y):
    """Berechnet die Negative Log-Likelihood."""
    z = steilheit * (X - mittelpunkt)
    p = sigmoid(z)
    # Kleine Epsilon-Werte um log(0) zu vermeiden
    eps = 1e-10
    p = np.clip(p, eps, 1 - eps)
    log_loss = -np.mean(Y * np.log(p) + (1 - Y) * np.log(1 - p))
    return log_loss

### Das Loss-Gebirge berechnen

Genau wie vorher berechnen wir den Loss für viele Kombinationen von Mittelpunkt und Steilheit.

In [ ]:
# Wertebereiche für Mittelpunkt und Steilheit
mittelpunkt_range = np.linspace(0, 6, 50)
steilheit_range = np.linspace(0.5, 10, 50)

# Gitter erstellen
MITTEL, STEIL = np.meshgrid(mittelpunkt_range, steilheit_range)

# Loss für jeden Punkt im Gitter berechnen
LOSS_KLASS = np.zeros_like(MITTEL)
for i in range(MITTEL.shape[0]):
    for j in range(MITTEL.shape[1]):
        LOSS_KLASS[i, j] = berechne_log_loss(MITTEL[i, j], STEIL[i, j], X_klass, Y_klass)

print("Loss-Gebirge berechnet!")
print(f"Minimaler Loss: {LOSS_KLASS.min():.4f}")

### Interaktive Visualisierung

Mit den Schiebereglern kannst du **Mittelpunkt** und **Steilheit** der Sigmoid-Kurve verändern.

- **Links**: Die Datenpunkte (blau = Klasse 0, orange = Klasse 1) und deine Sigmoid-Kurve
- **Rechts**: Das Loss-Gebirge mit deiner aktuellen Position

**Ziel**: Finde die Werte, bei denen der rote Punkt möglichst tief im Tal liegt!

In [ ]:
# Figure erstellen
fig2 = plt.figure(figsize=(12, 5))

# Linker Plot: Datenpunkte und Sigmoid-Kurve
ax3 = fig2.add_subplot(1, 2, 1)
ax3.set_xlim(-0.5, 6.5)
ax3.set_ylim(-0.1, 1.1)
ax3.set_xlabel('x')
ax3.set_ylabel('Wahrscheinlichkeit für Klasse 1')
ax3.set_title('Datenpunkte und Sigmoid-Kurve')
ax3.grid(True, alpha=0.3)

# Datenpunkte zeichnen (Farbe nach Label)
farben = ['blue' if y == 0 else 'orange' for y in Y_klass]
ax3.scatter(X_klass, Y_klass, c=farben, s=100, zorder=5, edgecolors='black')

# Legende manuell erstellen
ax3.scatter([], [], c='blue', s=100, edgecolors='black', label='Klasse 0')
ax3.scatter([], [], c='orange', s=100, edgecolors='black', label='Klasse 1')
ax3.legend()

# Sigmoid-Kurve (wird aktualisiert)
x_sigmoid = np.linspace(-0.5, 6.5, 200)
sigmoid_line, = ax3.plot(x_sigmoid, sigmoid(x_sigmoid), 'r-', linewidth=2)

# Vertikale Linie am Mittelpunkt
mittel_linie = ax3.axvline(x=3.0, color='gray', linestyle='--', alpha=0.5)

# Rechter Plot: Loss-Gebirge (3D)
ax4 = fig2.add_subplot(1, 2, 2, projection='3d')
surface2 = ax4.plot_surface(MITTEL, STEIL, LOSS_KLASS, cmap='coolwarm', alpha=0.8, edgecolor='none')
ax4.set_xlabel('Mittelpunkt')
ax4.set_ylabel('Steilheit')
ax4.set_zlabel('Loss')
ax4.set_title('Loss-Gebirge')

# Z-Boden für den Schatten: unteres Ende der Z-Achse
z_boden = 0

# Punkt-Projektion auf den Boden (Schatten) - SCHWARZ, zuerst zeichnen
bottom_point2, = ax4.plot([3.0], [1.0], [z_boden], 'ko', markersize=8, alpha=0.4)

# Verbindungslinie vom Schatten zum Punkt (gestrichelt)
init_loss = berechne_log_loss(3.0, 1.0, X_klass, Y_klass)
drop_line2, = ax4.plot([3.0, 3.0], [1.0, 1.0], [z_boden, init_loss], 'k--', linewidth=0.8, alpha=0.3)

# Aktueller Punkt im Loss-Gebirge (3D) - ROT, zuletzt zeichnen
current_point2, = ax4.plot([3.0], [1.0], [init_loss], 'ro', markersize=10, zorder=10)

# Loss-Anzeige als Text
loss_text2 = fig2.text(0.5, 0.02, '', ha='center', fontsize=12, fontweight='bold')

plt.tight_layout()

def update_klass(mittelpunkt, steilheit):
    """Aktualisiert die Plots wenn Schieberegler bewegt werden."""
    # Sigmoid-Kurve aktualisieren
    z = steilheit * (x_sigmoid - mittelpunkt)
    sigmoid_line.set_ydata(sigmoid(z))
    
    # Mittelpunkt-Linie aktualisieren
    mittel_linie.set_xdata([mittelpunkt, mittelpunkt])
    
    # Aktuellen Loss berechnen
    current_loss = berechne_log_loss(mittelpunkt, steilheit, X_klass, Y_klass)
    
    # Punkt im Loss-Gebirge aktualisieren (rot, 3D)
    current_point2.set_data_3d([mittelpunkt], [steilheit], [current_loss])
    
    # Verbindungslinie aktualisieren
    drop_line2.set_data_3d([mittelpunkt, mittelpunkt], [steilheit, steilheit], [z_boden, current_loss])
    
    # Projektion auf Boden aktualisieren (schwarz, 2D)
    bottom_point2.set_data_3d([mittelpunkt], [steilheit], [z_boden])
    
    # Loss-Text aktualisieren
    loss_text2.set_text(f'Aktueller Loss: {current_loss:.4f}  |  Mittelpunkt = {mittelpunkt:.2f}, Steilheit = {steilheit:.2f}')
    
    fig2.canvas.draw_idle()

# Schieberegler erstellen
mittelpunkt_slider = FloatSlider(
    value=3.0,
    min=0.0,
    max=6.0,
    step=0.1,
    description='Mittelpunkt:',
    continuous_update=True
)

steilheit_slider = FloatSlider(
    value=1.0,
    min=0.5,
    max=10.0,
    step=0.1,
    description='Steilheit:',
    continuous_update=True
)

# Interaktive Widgets anzeigen
interact(update_klass, mittelpunkt=mittelpunkt_slider, steilheit=steilheit_slider)

# Initiale Darstellung
update_klass(3.0, 1.0)
plt.show()

---

### Aufgabe: Finde die beste Trennung!

Experimentiere mit den Schiebereglern:

1. Bei welchem **Mittelpunkt** liegt die Grenze zwischen den Klassen am besten?
2. Wie wirkt sich die **Steilheit** auf die Vorhersage aus?
3. Was passiert mit dem Loss, wenn die Sigmoid-Kurve komplett falsch liegt (z.B. Mittelpunkt = 0)?

**Deine Antworten:**

(Hier doppelklicken zum Bearbeiten)

- Bester Mittelpunkt: 
- Beste Steilheit: 
- Minimaler Loss: 
- Was passiert bei falschem Mittelpunkt?: 

---

## Teil 3: Das gleiche mit scikit-learn

Bisher haben wir alles von Hand programmiert: Loss-Funktionen, Optimierung, Plots. Das ist super, um zu verstehen, was im Hintergrund passiert!

In der Praxis nutzt man aber fast immer eine **Bibliothek**, die das alles für uns erledigt. Die bekannteste in Python heißt **scikit-learn** (sprich: "sei-kit lörn").

### Das Grundprinzip: `.fit()` und `.predict()`

Scikit-learn funktioniert immer nach dem gleichen Muster -- egal ob lineare Regression, logistische Regression, Entscheidungsbäume, neuronale Netze oder hundert andere Verfahren:

```python
# 1. Modell erstellen
modell = IrgendeinModell()

# 2. Modell an Daten anpassen (= "trainieren")
modell.fit(X_train, Y_train)

# 3. Vorhersagen machen
vorhersagen = modell.predict(X_neu)
```

**Das war's.** Drei Zeilen statt vieler Dutzend.

- **`.fit()`** macht das, was wir oben von Hand gemacht haben: Es sucht die besten Parameter (slope, intercept, Mittelpunkt, Steilheit...), sodass der Loss minimal wird.
- **`.predict()`** benutzt die gefundenen Parameter, um Vorhersagen für neue Daten zu machen.

Das Schöne: Wenn du dieses Muster einmal verstanden hast, kannst du fast jedes ML-Verfahren in scikit-learn benutzen. Die API ist immer gleich.

### Lineare Regression mit scikit-learn

Erinnere dich: Oben haben wir die optimale Gerade von Hand berechnet (Normalengleichung) und sind auf slope = 1.8 und intercept = 0.5 gekommen. Schauen wir mal, ob scikit-learn das gleiche findet.

**Wichtig:** Scikit-learn erwartet die X-Daten als 2D-Array (Tabelle), nicht als einfache Liste. Das liegt daran, dass man in der Praxis oft mehrere Features (Spalten) hat. Wir haben nur ein Feature, also machen wir aus unserem 1D-Array ein 2D-Array mit einer Spalte -- das erledigt `.reshape(-1, 1)`.

In [ ]:
from sklearn.linear_model import LinearRegression

# Schritt 1: Modell erstellen
lin_modell = LinearRegression()

# Schritt 2: Modell trainieren (fit)
# reshape(-1, 1) macht aus [1, 2, 3, 4] eine "Tabelle" mit einer Spalte:
#   [[1],
#    [2],
#    [3],
#    [4]]
lin_modell.fit(X.reshape(-1, 1), Y)

# Was hat scikit-learn gefunden?
print("Scikit-learn hat folgende Parameter gefunden:")
print(f"  slope     = {lin_modell.coef_[0]:.4f}")
print(f"  intercept = {lin_modell.intercept_:.4f}")
print()
print("Vergleich mit unserer Normalengleichung:")
print(f"  slope     = {slope_optimal:.4f}  (identisch!)")
print(f"  intercept = {intercept_optimal:.4f}  (identisch!)")

Exakt die gleichen Werte! Scikit-learn hat im Hintergrund genau das gemacht, was wir vorher von Hand berechnet haben.

Jetzt können wir mit `.predict()` Vorhersagen machen -- auch für x-Werte, die gar nicht in unseren Daten vorkommen:

In [ ]:
# Schritt 3: Vorhersagen machen (predict)
x_neu = np.array([0, 1.5, 2.5, 5, 10]).reshape(-1, 1)
vorhersagen = lin_modell.predict(x_neu)

print("Vorhersagen für neue x-Werte:")
for xi, yi in zip(x_neu.flatten(), vorhersagen):
    print(f"  x = {xi:5.1f}  -->  y = {yi:.2f}")

# Visualisierung
fig3, ax5 = plt.subplots(figsize=(8, 5))

# Originale Datenpunkte
ax5.scatter(X, Y, color='blue', s=100, zorder=5, label='Trainingsdaten')

# Gerade von scikit-learn
x_plot = np.linspace(0, 5, 100).reshape(-1, 1)
y_plot = lin_modell.predict(x_plot)
ax5.plot(x_plot, y_plot, 'r-', linewidth=2, label='scikit-learn Gerade')

# Neue Vorhersagen hervorheben
ax5.scatter(x_neu[:-1], vorhersagen[:-1], color='green', s=120, zorder=5,
            marker='*', label='Vorhersagen')

ax5.set_xlabel('x')
ax5.set_ylabel('y')
ax5.set_title('Lineare Regression mit scikit-learn')
ax5.legend()
ax5.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Logistische Regression mit scikit-learn

Jetzt das gleiche Spiel für die Klassifikation. Wir benutzen die gleichen Daten (`X_klass`, `Y_klass`) von oben und lassen scikit-learn die beste Sigmoid-Kurve finden.

Der Code sieht fast identisch aus -- nur statt `LinearRegression` nehmen wir `LogisticRegression`:

In [ ]:
from sklearn.linear_model import LogisticRegression

# Schritt 1: Modell erstellen
# penalty=None schaltet die "Regularisierung" aus -- damit findet scikit-learn
# genau die gleiche Lösung, die wir oben mit den Schiebereglern gesucht haben.
# (Regularisierung ist ein fortgeschrittenes Thema für später.)
log_modell = LogisticRegression(penalty=None)

# Schritt 2: Modell trainieren (fit) -- gleiche Daten wie oben!
log_modell.fit(X_klass.reshape(-1, 1), Y_klass)

# Die Parameter, die scikit-learn gefunden hat
sk_steilheit = log_modell.coef_[0][0]
sk_mittelpunkt = -log_modell.intercept_[0] / sk_steilheit

print("Scikit-learn hat folgende Parameter gefunden:")
print(f"  Steilheit   = {sk_steilheit:.4f}")
print(f"  Mittelpunkt = {sk_mittelpunkt:.4f}")
print()
print(f"Zur Erinnerung: Die Daten wurden mit Mitte={true_mitte}, Steilheit={true_steilheit} generiert.")

Jetzt nutzen wir `.predict()` um Klassen vorherzusagen, und `.predict_proba()` um die Wahrscheinlichkeiten zu sehen:

In [ ]:
# Schritt 3: Vorhersagen machen
x_test = np.array([1.0, 2.5, 3.0, 3.5, 5.0]).reshape(-1, 1)

# .predict() gibt die Klasse (0 oder 1)
klassen = log_modell.predict(x_test)

# .predict_proba() gibt die Wahrscheinlichkeiten für jede Klasse
wahrscheinlichkeiten = log_modell.predict_proba(x_test)

print("Vorhersagen für neue x-Werte:")
print(f"  {'x':>5}  {'Klasse':>6}  {'P(Klasse 0)':>12}  {'P(Klasse 1)':>12}")
print(f"  {'─'*5}  {'─'*6}  {'─'*12}  {'─'*12}")
for xi, ki, wi in zip(x_test.flatten(), klassen, wahrscheinlichkeiten):
    print(f"  {xi:5.1f}  {ki:6d}  {wi[0]:12.1%}  {wi[1]:12.1%}")

In [ ]:
# Visualisierung: Datenpunkte + scikit-learn Sigmoid-Kurve
fig4, ax6 = plt.subplots(figsize=(8, 5))

# Datenpunkte (Farbe nach Label)
farben = ['blue' if y == 0 else 'orange' for y in Y_klass]
ax6.scatter(X_klass, Y_klass, c=farben, s=100, zorder=5, edgecolors='black')

# Legende
ax6.scatter([], [], c='blue', s=100, edgecolors='black', label='Klasse 0')
ax6.scatter([], [], c='orange', s=100, edgecolors='black', label='Klasse 1')

# Sigmoid-Kurve von scikit-learn
x_plot_sig = np.linspace(-0.5, 6.5, 200).reshape(-1, 1)
p_plot = log_modell.predict_proba(x_plot_sig)[:, 1]  # Wahrscheinlichkeit für Klasse 1
ax6.plot(x_plot_sig, p_plot, 'r-', linewidth=2, label='scikit-learn Sigmoid')

# Entscheidungsgrenze (wo P = 0.5)
ax6.axvline(x=sk_mittelpunkt, color='gray', linestyle='--', alpha=0.5,
            label=f'Grenze bei x={sk_mittelpunkt:.2f}')
ax6.axhline(y=0.5, color='gray', linestyle=':', alpha=0.3)

ax6.set_xlabel('x')
ax6.set_ylabel('Wahrscheinlichkeit für Klasse 1')
ax6.set_title('Logistische Regression mit scikit-learn')
ax6.legend(loc='center right')
ax6.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Zusammenfassung

Was wir gelernt haben:

| | Von Hand | Mit scikit-learn |
|---|---|---|
| **Lineare Regression** | Normalengleichung, ~10 Zeilen | `LinearRegression()` + `.fit()` + `.predict()` |
| **Logistische Regression** | Loss-Funktion + Schieberegler, ~30 Zeilen | `LogisticRegression()` + `.fit()` + `.predict()` |

Das Muster ist immer gleich:
1. **Modell erstellen** -- z.B. `LinearRegression()` oder `LogisticRegression()`
2. **`.fit(X, Y)`** -- das Modell lernt aus den Daten
3. **`.predict(X_neu)`** -- das Modell macht Vorhersagen

Dieses Prinzip funktioniert genauso für Entscheidungsbäume, Random Forests, Support Vector Machines und viele weitere Verfahren in scikit-learn. Wenn du `.fit()` und `.predict()` verstanden hast, kannst du sie alle benutzen!